## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

In [28]:
import os
import sys
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.model_selection import train_test_split
from collections import Counter
import re
import torch
from torch.utils.data import Dataset, DataLoader

In [29]:
# Global variables

# Vocabulary
vocab_size = 3000
context_size = 9
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 32
epochs = 10
lr = 0.001

# Model
embedding_dim = 64
hidden_dim = 128

In [30]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

## Faz download e carrega o dataset

In [31]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [32]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [33]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [34]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

Ora dizei-me, Sr. cavalheiro, não vos parece claro, á vista do que me ouvistes, que adevinhei o vosso desejo de chegar o mais depressa possivel?

Ayres Gomes quiz retrucar; mas a dama impoz-lhe silencio com um gesto.

A esposa do prsioneiro atravessou o campo e offereceu ao vencedor um grande vaso de barro vidrado cheio de vinho de ananaz ainda espumante.

A canôa resvalou pela flôr d'agua, como uma garça ligeira levada pela correnteza do rio.

O BRACELETE

Number of paragraphs: 4892


4892

## Análise do dataset

In [35]:
# Conta as palavras no dataset
def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [36]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [37]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 3000


In [38]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 0, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [39]:
# Dataset class
class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [40]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 24194
Validation dataset samples: 6815


In [55]:
tst_loader = DataLoader(train_dataset, batch_size = 1, shuffle=True)
sample = next(iter(tst_loader))
print(sample)

[tensor([[  2, 606,  18,   3,  68,   2, 238,  35,  96]]), tensor([159])]


In [41]:

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

[tensor([[  16,    5,  972,   34,  528,   30,   61,  108,  225],
        [  64, 2856,   11,  510,   16,    5,  215,   36, 1196],
        [   2,  305,   10,   16,   22,   95,  277,    2,  305],
        [   6, 1805, 1952,    2, 2087,   27,  216,    2,   36],
        [ 134,   61,  143,    6,   17,   57,    2,    7,   12],
        [  17,  396,    2,  519,    3,   99,   38,    3,  615],
        [ 323,   89,  220,    5,  400,   11,   87, 1345, 2953],
        [ 345,    6, 2088,    2,    6, 1470,  526,    2,   23],
        [   1,   37,  251,    4,  474,  648,   17,  175,    1],
        [1100,  813,    6,  596,    1,   21,  712,    5,  436],
        [   7, 1362,    2, 1333,   23,   13, 1307,   24,   91],
        [  53,   41,   74,   66,  402, 1945,  148,  151,  544],
        [ 145,   31,  157,    2,   93,   18,   30,    1,   47],
        [  27,  762,    6,   97,   28,   88,    9,   50,    5],
        [   6,   30,    1,   47,    6,   73,    5,  102, 1073],
        [ 231,  620,    9,   10,  575, 

## Model

In [42]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        #self.relu = torch.nn.ReLU()
        # Tanh non-linearity as described in the article
        self.tanh = torch.nn.Tanh()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.tanh(out)
        #out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

In [43]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

In [44]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

torch.Size([32, 9])
torch.Size([32])


In [45]:
output = model(input)
print(output)
print(output.shape)

tensor([[-7.9693, -7.9946, -7.7639,  ..., -8.3126, -8.0013, -8.1966],
        [-7.6328, -8.2133, -7.9409,  ..., -8.3627, -8.0449, -8.1198],
        [-7.1843, -8.2905, -7.7408,  ..., -7.9079, -8.0930, -8.0568],
        ...,
        [-8.1622, -8.1172, -7.9562,  ..., -8.3005, -8.2247, -8.4828],
        [-8.1401, -8.0558, -7.8546,  ..., -8.3732, -8.0077, -7.9608],
        [-8.2382, -8.0458, -8.0519,  ..., -7.6923, -7.6369, -7.9338]],
       grad_fn=<LogSoftmaxBackward0>)
torch.Size([32, 3001])


In [46]:
output.argmax(dim=1)

tensor([2359, 2838, 1152,  397,  295,  955, 1659, 1688, 1583, 1416, 2142, 1709,
        1660, 1196,  298, 2342, 1375, 1010, 1349, 2030,  700, 1111, 1624, 2408,
        2670, 2575, 1806, 2437,  944,   41, 2257,  596])

In [47]:
target

tensor([   7, 2156,   14,    1,   93,  105,    5,    1,    1,    1,  926,   40,
         810,   15,   14,   29,   98,    2,    1,   23,   15,  322, 2436,   91,
         217,    1,  590,    1,   41,    8, 1865,    4])

## Training

In [48]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [49]:
# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr)
#optimizer = torch.optim.SGD(model.parameters(), lr)

model.to(device)

BengioModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=576, out_features=128, bias=True)
  (tanh): Tanh()
  (linear2): Linear(in_features=128, out_features=3001, bias=True)
  (logSoftMax): LogSoftmax(dim=1)
)

In [50]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 653,049 parâmetros.


In [51]:
# Initial Perplexity and Loss
# Before training
model.eval()

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

loss /= len(train_loader)
perp = torch.exp(torch.tensor(loss))

print(f'Initial Loss: {loss:.4f}')
print(f'Initial Perplexity: {perp:.4f}')

Initial Loss: 8.0538
Initial Perplexity: 3145.8240


In [52]:
# Training Loop
model.train()
for epoch in range(epochs):

  epoch_start = time.time()
  # Metrics
  epoch_loss = 0
  epoch_correct = 0
  epoch_samples = 0

  for inputs, targets in train_loader:
        inputs = inputs.to(device)  # Move input data to the device
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss
        epoch_loss += loss.item()

        # Predicted
        _, predicted = torch.max(outputs, 1)
        epoch_correct += (predicted == targets).sum().item()
        epoch_samples += targets.size(0)

  # Calculate average loss and accuracy for epoch
  avg_loss = epoch_loss / len(train_loader)
  acc = epoch_correct / epoch_samples

  # Perplexity
  perp = torch.exp(torch.tensor(avg_loss))

  epoch_end = time.time()
  epoch_time = epoch_end - epoch_start
  # Print epoch statistics
  print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


Epoch [1/50], Time:15.58, Loss: 5.9224, Accuracy: 0.12%, Perplexity: 373.3233
Epoch [2/50], Time:12.91, Loss: 4.7200, Accuracy: 0.19%, Perplexity: 112.1685
Epoch [3/50], Time:13.73, Loss: 4.0067, Accuracy: 0.24%, Perplexity: 54.9648
Epoch [4/50], Time:13.98, Loss: 3.3971, Accuracy: 0.30%, Perplexity: 29.8779
Epoch [5/50], Time:14.62, Loss: 2.8320, Accuracy: 0.38%, Perplexity: 16.9798
Epoch [6/50], Time:14.54, Loss: 2.3235, Accuracy: 0.49%, Perplexity: 10.2114
Epoch [7/50], Time:13.25, Loss: 1.8838, Accuracy: 0.59%, Perplexity: 6.5784
Epoch [8/50], Time:13.22, Loss: 1.5215, Accuracy: 0.68%, Perplexity: 4.5791
Epoch [9/50], Time:13.07, Loss: 1.2164, Accuracy: 0.75%, Perplexity: 3.3749
Epoch [10/50], Time:13.39, Loss: 0.9662, Accuracy: 0.81%, Perplexity: 2.6278
Epoch [11/50], Time:12.44, Loss: 0.7600, Accuracy: 0.86%, Perplexity: 2.1384
Epoch [12/50], Time:17.23, Loss: 0.5901, Accuracy: 0.90%, Perplexity: 1.8041
Epoch [13/50], Time:13.64, Loss: 0.4457, Accuracy: 0.93%, Perplexity: 1.5616


## Avaliação

In [53]:
model.eval()

loss_sum = 0
total_sum = 0
correct_sum = 0
eval_round = 0

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)      
        loss_sum += loss

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        total_sum += targets.size(0)
        correct_sum += (predicted == targets).sum().item()
        eval_round += 1

# Calculate accuracy
acc = 100 * correct_sum / total_sum

# Calculate average perplexity
average_loss = loss_sum / len(val_loader)
average_perplexity = torch.exp(average_loss)

print(f'Test Accuracy: {acc:.2f}%')
print(f'Average Loss: {average_loss:.2f}')
print(f'Average Perplexity: {average_perplexity:.2f}')

Test Accuracy: 25.22%
Average Loss: 9.83
Average Perplexity: 18555.04


## Exemplo de uso

In [54]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")


max_length= 10
generate_text(model, vocab, text, max_length, context_size)

Frase: Adivinhava e comprehendia o que sentia então o coração de seus bons amigos; sabia que soffrião vendo-o prisioneiro, proximo a morrer, sem terem o poder e a força para salva-lo das mãos do inimigo.
[1845, 7, 925, 5, 4, 254, 101, 5, 141]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25, 2]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25, 2, 344]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25, 2, 344, 4]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25, 2, 344, 4, 8]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25, 2, 344, 4, 8, 6]
[1845, 7, 925, 5, 4, 254, 101, 5, 141, 12, 29, 648, 25, 2, 344, 4, 8, 6, 137]
['adivinhava', 'e', 'comprehendia', 'o', 'que', 'sentia', 'então', 'o', 'coração'] ['não', 'lhe', 'estar', '?', ',', 'apezar', 'que', 'se', 'de', 'd